In [ ]:
"""
Why am I doing s_norm[:, :, 1] *= -1 in process panel

file:
/net/dials/raid1/sauter/bernina/spectrum_masters/run_000795.JF07T32V01_master.h5

Vanessa converted the first frame to a cbf and put it at
/net/cci/voklejas/repos/dials/build/dials_data/converted.cbf

Possible utilities
    /home/david/dials_dev/modules/dials/util/image_viewer/slip_viewer/tile_generation.py
"""

In [ ]:
import sys
sys.path.append('/home/david/dials_dev/modules/dxtbx/src')
sys.path.append('/home/david/dials_dev/build/lib')
sys.path.append('/home/david/dials_dev/modules')
from cctbx import factor_kev_angstrom
from dials.array_family import flex
import dxtbx
#from dxtbx.model.experiment_list import ExperimentListFactory
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

import Modules_Multipanel as MOD


mpl.rc_file('matplotlibrc.txt')

In [ ]:
image_file_name = '/home/david/Documents/Background/Data/converted.cbf'
image = dxtbx.load(image_file_name)
beam = image.get_beam()
wavelength = beam.get_wavelength()
data = image.get_raw_data()
detector = image.get_detector()
pixel_size = detector[0].get_pixel_size()[0]
kapton_absorption_length = MOD.get_absorption_correction()(wavelength)

In [ ]:
polarization_fraction = 1
I, s, s_norm, mask, theta2, phi, polarization, integrated_image =\
    MOD.GetImage(data, detector, polarization_fraction)
normalized_image = I / (polarization * integrated_image)

In [ ]:
angle = 275 * np.pi/180
h = 0.04
f = 0.665
t = 0.025
f_range = 0.1
absorption = MOD.IntegrateModel(angle, h, f, t, s_norm, kapton_absorption_length, f_range, n=3)
absorption[mask] = -1

In [ ]:
def Fit(params, t, normalized_image, s_norm, mask, kapton_absorption_length):
    angle = params[0]
    h = params[1]
    f = params[2]
    absorption = MOD.IntegrateModel(angle, h, f, t, s_norm, kapton_absorption_length, 0.1, n=3)
    residuals = (normalized_image - absorption)[np.invert(mask)]
    print(angle * 180/np.pi)
    print(h)
    print(f)
    print(np.linalg.norm(residuals))
    print()
    return np.linalg.norm(residuals)

In [ ]:
def ProcessPanelFlat(data, panel):
    I = data.as_numpy_array()
    panel_shape = panel.get_image_size()
    x, y = np.meshgrid(
        np.linspace(0, panel_shape[0] - 1, panel_shape[0]),
        np.linspace(0, panel_shape[1] - 1, panel_shape[1])
        )
    mm = panel.pixel_to_millimeter(flex.vec2_double(
        flex.double(x.flatten()),
        flex.double(y.flatten())
        ))
    s = panel.get_lab_coord(mm).as_numpy_array()
    s_norm = (s.T / np.linalg.norm(s, axis=1)).T
    s_norm[:, 2] *= -1  
    return I, s, s_norm

In [ ]:
def GetImageFlat(data, detector, polarization_fraction):
    I = np.zeros(16*16*254*254)
    s = np.zeros((16*16*254*254, 3))
    s_norm = np.zeros((16*16*254*254, 3))
    mask = np.zeros(16*16*254*254, dtype=np.bool)
    for index in range(16*16):
        I_panel, s_panel, s_norm_panel = ProcessPanelFlat(data[index], detector[index])
        start = index * 254*254
        end = (index + 1) * 254*254
        I[start: end] = I_panel
        s[start: end] = s_panel
        s_norm[start: end] = s_norm_panel
    mask = np.logical_or(
        mask,
        I <= 0
        )
    R = np.sqrt(s[:, 0]**2 + s[:, 1]**2)
    theta2 = np.arctan(R / np.abs(s[:, 2]))
    phi = np.pi + np.arctan2(s[:, 1], s[:, 0])
    polarization = (1 - polarization_fraction*np.cos(2*phi)*np.sin(theta2)**2 / (1+np.cos(theta2)**2))
    theta2_int = np.linspace(0, 60, 121) * np.pi/180
    integratation_indices = np.invert(mask)

    integration_sum = np.histogram(
        theta2[integratation_indices].flatten(),
        bins=theta2_int,
        weights=I[integratation_indices].flatten()
        )
    integration_counts = np.histogram(
        theta2[integratation_indices].flatten(),
        bins=theta2_int
        )
    bins = integration_counts[1]
    bin_centers = (bins[1:] + bins[:-1])/2
    integrated = integration_sum[0] / integration_counts[0]
    indices = np.invert(np.isnan(integrated))
    integrated_image = np.interp(theta2, bin_centers[indices], integrated[indices])
    integrated_image[mask] = -1
    return I, s, s_norm, mask, theta2, phi, polarization, integrated_image

In [ ]:
results_fit = minimize(
    Fit,
    x0=(angle, h, f),
    args=(t, normalized_image, s_norm, mask, kapton_absorption_length),
    method='L-BFGS-B',
    bounds=((0, 2*np.pi), (0.001, None), (0.001, None))
    )
angle_fit = results_fit.x[0]
h_fit = results_fit.x[1]
f_fit = results_fit.x[2]
print(results_fit)

In [ ]:
D = detector[0].get_distance()
delta = D * (h_fit + t) / f_fit
alpha2 = angle_fit - 3/2 * np.pi
b = delta / np.cos(alpha2)
m = np.tan(alpha2)
 
pixel_size = detector[0].get_pixel_size()[0]
min_pos = np.zeros(2)
max_pos = np.zeros(2)
for index in range(16*16):
    origin = detector[index].get_origin()
    if origin[0] < min_pos[0]:
        min_pos[0] = origin[0]
    if origin[1] < min_pos[1]:
        min_pos[1] = origin[1]
    if origin[0] > max_pos[0]:
        max_pos[0] = origin[0]
    if origin[1] > max_pos[1]:
        max_pos[1] = origin[1]

max_pos += 256 * pixel_size
y = np.array((0, max_pos[1] - min_pos[1]))
y_pix = y / pixel_size
b_pix = b / pixel_size
x = m * (y + min_pos[1]) - b - min_pos[0]
"""
fig, axes = plt.subplots(1, 1, figsize=(20, 20))
axes.imshow(absorption)
axes.plot(x / pixel_size, y / pixel_size)
plt.show()
"""
xx, yy = np.meshgrid(
    np.linspace(0, I.shape[1]*pixel_size, I.shape[1]),
    np.linspace(0, I.shape[0]*pixel_size, I.shape[0])
    )
delta_x = (xx - (m * (yy + min_pos[1]) - b - min_pos[0])) * np.cos(alpha2)

"""
fig, axes = plt.subplots(1, 1, figsize=(20, 20))
axes.imshow(delta_x)
axes.plot(x / pixel_size, y / pixel_size)
plt.show()
"""

In [ ]:
delta_x_int = np.linspace(delta_x.min(), delta_x.max(), 100)
bin_centers = (delta_x_int[1:] + delta_x_int[:-1]) / 2

normalized_sum = np.histogram(
    delta_x[np.invert(mask)].flatten(),
    bins=delta_x_int,
    weights=normalized_image[np.invert(mask)].flatten()
    )
absorption_sum = np.histogram(
    delta_x[np.invert(mask)].flatten(),
    bins=delta_x_int,
    weights=absorption[np.invert(mask)].flatten()
    )
counts = np.histogram(
    delta_x[np.invert(mask)].flatten(),
    bins=delta_x_int
    )
averaged_absorption = absorption_sum[0] / counts[0]
averaged_normalized = normalized_sum[0] / counts[0]

fig, axes = plt.subplots(1, 1)
axes.plot(bin_centers, averaged_normalized)
axes.plot(bin_centers, averaged_absorption, linewidth=1)
plt.show()

absorption = MOD.IntegrateModel(angle_fit, h_fit, f_fit, t, s_norm, kapton_absorption_length, f_range, n=3)
absorption[mask] = -1


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 20))
axes[0].imshow(I / (polarization * integrated_image), vmin=0.8, vmax=1.2)
axes[1].imshow(absorption, vmin=0.85, vmax=1)
fig, axes = plt.subplots(1, 2, figsize=(20, 20))
axes[0].imshow(normalized_image, vmin=0.9, vmax=1.1)
axes[1].imshow(normalized_image / absorption, vmin=0.9, vmax=1.1)
plt.show()


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(20, 20))
axes[0].imshow(I, vmin=0, vmax=600)
axes[1].imshow(polarization)
axes[2].imshow(integrated_image)
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(20, 20))
axes[0].imshow(I / polarization, vmin=0, vmax=600)
axes[1].imshow(I / integrated_image, vmin=0.75, vmax=1.25)
axes[2].imshow(I / (polarization * integrated_image), vmin=0.8, vmax=1.2)
plt.show()